In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
import os

train_path = "https://media.githubusercontent.com/media/PavelKilko/applied-application-of-neural-networks/refs/heads/master/lab-6-7/small-dataset/train.csv"
val_path = "https://media.githubusercontent.com/media/PavelKilko/applied-application-of-neural-networks/refs/heads/master/lab-6-7/small-dataset/val.csv"
test_path = "https://media.githubusercontent.com/media/PavelKilko/applied-application-of-neural-networks/refs/heads/master/lab-6-7/small-dataset/train.csv"

def preprocess_data(train_path, val_path, test_path):
    train_dataset = pd.read_csv(train_path)
    val_dataset = pd.read_csv(val_path)
    test_dataset = pd.read_csv(test_path)

    X_train = train_dataset.drop('font', axis=1).astype('float32').values
    y_train = train_dataset['font'].values

    X_val = val_dataset.drop('font', axis=1).astype('float32').values
    y_val = val_dataset['font'].values

    X_test = test_dataset.drop('font', axis=1).astype('float32').values
    y_test = test_dataset['font'].values

    X_train = X_train.reshape(-1, 20, 20, 1)
    X_val = X_val.reshape(-1, 20, 20, 1)
    X_test = X_test.reshape(-1, 20, 20, 1)

    unique_labels = np.unique(y_train)
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    y_train_encoded = np.array([label_to_index[label] for label in y_train])
    y_val_encoded = np.array([label_to_index[label] for label in y_val])
    y_test_encoded = np.array([label_to_index[label] for label in y_test])

    num_classes = len(unique_labels)

    y_train = to_categorical(y_train_encoded, num_classes=num_classes)
    y_val = to_categorical(y_val_encoded, num_classes=num_classes)
    y_test = to_categorical(y_test_encoded, num_classes=num_classes)

    return X_train, y_train, X_val, y_val, X_test, y_test, num_classes, unique_labels

X_train, y_train, X_val, y_val, X_test, y_test, num_classes, unique_labels = preprocess_data(train_path, val_path, test_path)

# Store the model globally so it can be saved on demand
trained_model = None

# Build and train model with user-defined parameters
def build_and_train_model(filter1, filter2, filter3, kernel1, kernel2, kernel3, dense_neurons, learning_rate, dropout, patience_es, factor_rlr, patience_rlr, min_lr, epochs):
    global trained_model
    model = Sequential()

    model.add(Input(shape=(20, 20, 1)))

    model.add(Conv2D(filter1, (kernel1, kernel1), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filter1, (kernel1, kernel1), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(filter2, (kernel2, kernel2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filter2, (kernel2, kernel2), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(filter3, (kernel3, kernel3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filter3, (kernel3, kernel3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(dense_neurons, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_rlr, patience=patience_rlr, min_lr=min_lr)

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        validation_data=(X_val, y_val),
        batch_size=128,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Store the trained model globally
    trained_model = model

    # Plot training results
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Training and Validation Metrics')
    plt.savefig('training_results.png')

    return f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}", 'training_results.png'

# Save model on demand
def save_model():
    global trained_model
    if trained_model is None:
        return None
    model_path = "trained_model.h5"
    trained_model.save(model_path)
    return model_path

# Gradio Interface
interface = gr.Interface(
    fn=build_and_train_model,
    inputs=[
        gr.Number(label="Filter Number (1st Layer)", value=32),
        gr.Number(label="Filter Number (2nd Layer)", value=64),
        gr.Number(label="Filter Number (3rd Layer)", value=128),
        gr.Number(label="Kernel Size (1st Layer)", value=3),
        gr.Number(label="Kernel Size (2nd Layer)", value=3),
        gr.Number(label="Kernel Size (3rd Layer)", value=3),
        gr.Number(label="Dense Layer Neurons", value=256),
        gr.Number(label="Learning Rate", value=0.0005),
        gr.Number(label="Dropout Value", value=0.3),
        gr.Number(label="Early Stopping Patience", value=15),
        gr.Number(label="Reduce LR Factor", value=0.2),
        gr.Number(label="Reduce LR Patience", value=7),
        gr.Number(label="Min LR", value=1e-6),
        gr.Number(label="Epochs", value=100)
    ],
    outputs=[
        gr.Textbox(label="Final Results"),
        gr.Image(label="Training Results Plot")
    ]
)

# Separate interface for downloading the model
download_interface = gr.Interface(
    fn=save_model,
    inputs=[],
    outputs=gr.File(label="Download Trained Model"),
    live=False
)

# Combine both interfaces
gr.TabbedInterface([interface, download_interface], ["Train Model", "Download Model"]).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://34e69349e42f360e08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


244/244 ━━━━━━━━━━━━━━━━━━━━ 144s 561ms/step - accuracy: 0.1713 - loss: 3.8778 - val_accuracy: 0.3576 - val_loss: 2.7165 - learning_rate: 5.0000e-04
Epoch 1/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 146s 565ms/step - accuracy: 0.1741 - loss: 3.8878 - val_accuracy: 0.3465 - val_loss: 2.6925 - learning_rate: 5.0000e-04
Epoch 2/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 134s 548ms/step - accuracy: 0.3282 - loss: 2.7981 - val_accuracy: 0.4020 - val_loss: 2.3693 - learning_rate: 5.0000e-04
Epoch 3/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 149s 579ms/step - accuracy: 0.3897 - loss: 2.5030 - val_accuracy: 0.4350 - val_loss: 2.2204 - learning_rate: 5.0000e-04


In [16]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps
import requests
import json
import matplotlib.pyplot as plt


# Constants for model and labels URLs
MODEL_URL = "https://github.com/PavelKilko/applied-application-of-neural-networks/raw/refs/heads/master/lab-9-10/small-font-recognizer-cnn-v1.1.h5"  # Replace with the actual model URL
LABELS_URL = "https://raw.githubusercontent.com/PavelKilko/applied-application-of-neural-networks/refs/heads/master/lab-9-10/unique_labels.json"  # Replace with the actual labels JSON URL


# Helper function to download model and labels
def download_model_and_labels(model_url, labels_url):
    # Download the model
    model_path = "model.h5"
    with open(model_path, "wb") as f:
        f.write(requests.get(model_url).content)

    # Download the labels
    labels_path = "labels.json"
    with open(labels_path, "wb") as f:
        f.write(requests.get(labels_url).content)

    # Load model and labels
    try:
        model = tf.keras.models.load_model(model_path)
    except ValueError as e:
        raise RuntimeError("Failed to load the model. Ensure the .h5 file is valid and matches the input shape.") from e

    with open(labels_path, "r") as f:
        unique_labels = json.load(f)

    return model, unique_labels


# Helper function to process the image
def preprocess_image(image):
    # Convert the image to grayscale
    grayscale_image = ImageOps.grayscale(image)

    # Resize the image to the nearest multiple of 20 (if not already divisible)
    width, height = grayscale_image.size
    width -= width % 20
    height -= height % 20
    grayscale_image = grayscale_image.resize((width, height))

    # Split into 20x20 squares
    squares = []
    for i in range(0, height, 20):
        for j in range(0, width, 20):
            square = grayscale_image.crop((j, i, j + 20, i + 20))
            square_array = np.array(square).astype('float32') / 255.0
            square_array = square_array.reshape(20, 20, 1)  # Add channel dimension
            squares.append(square_array)

    return np.array(squares), width // 20, height // 20, grayscale_image


# Prediction function
def predict(image):
    # Download model and labels
    model, unique_labels = download_model_and_labels(MODEL_URL, LABELS_URL)

    # Preprocess the image
    squares, grid_width, grid_height, grayscale_image = preprocess_image(image)

    # Make predictions for each square
    try:
        predictions = model.predict(squares)
    except ValueError as e:
        raise RuntimeError("Failed to predict. Ensure the model input shape matches (20, 20, 1).") from e

    # Update to handle list-based unique_labels
    predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

    # Visualize results
    # Visualize results
    fig, ax = plt.subplots(grid_height, grid_width, figsize=(15, 15))

    # Ensure ax is a 2D array
    if grid_width == 1 and grid_height == 1:
        ax = np.array([[ax]])  # Convert single Axes object to a 2D array
    elif grid_width == 1 or grid_height == 1:
        ax = np.expand_dims(ax, axis=0 if grid_height == 1 else 1)

    for i in range(grid_height):
        for j in range(grid_width):
            square_idx = i * grid_width + j
            ax[i, j].imshow(squares[square_idx].reshape(20, 20), cmap='gray')
            ax[i, j].axis('off')
            ax[i, j].set_title(predicted_labels[square_idx], fontsize=16)  # Increased font size

    # Save the result visualization
    result_path = "predictions.png"
    plt.tight_layout()
    plt.savefig(result_path)
    plt.close(fig)  # Ensure the figure is closed to avoid memory issues

    return result_path


# Gradio Interface
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(label="Upload Image", type="pil")
    ],
    outputs=gr.Image(label="Predicted Squares with Labels"),
    live=False
)

interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eb75cb5e763fb1ada4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://eb75cb5e763fb1ada4.gradio.live
